In [86]:
# importing the Dataset

import pandas as pd
import re
import nltk
import numpy as np

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

df = pd.read_csv('SMSSpamCollection', sep='\t',
                           names=["label", "message"])

messages = df.iloc[:,1]

In [14]:
messages=messages.values

In [15]:
type(messages)

numpy.ndarray

In [5]:
messages[0]

'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'

In [16]:
messages.shape

(5572,)

In [17]:
len(messages)

5572

In [20]:
def preprocess(messages):
    wordnet=WordNetLemmatizer()
    corpus = []
    for i in range(0, len(messages)):
        review = re.sub('[^a-zA-Z]', ' ', messages[i])
        review = review.lower()
        review = word_tokenize(review)

        review = [wordnet.lemmatize(word) for word in review if not word in stopwords.words('english')]
        review = ' '.join(review)
        corpus.append(review)
    return corpus
    
    

corpus = preprocess(messages)

In [23]:
len(corpus)

5572

In [24]:
# Creating the TF-IDF model
from sklearn.feature_extraction.text import TfidfVectorizer
cv = TfidfVectorizer()
cv.fit(corpus)

X = cv.transform(corpus).toarray()

y=pd.get_dummies(df['label']) # creates two columns one for ham and one for spam
y=y.iloc[:,1].values # we are keeping only spam column as it will be enough to tell if the message is spam or ham

In [25]:
X.shape

(5572, 7094)

In [26]:
X

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [27]:

# Train Test Split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

# Training model using Naive bayes classifier
from sklearn.naive_bayes import MultinomialNB
spam_detect_model = MultinomialNB().fit(X_train, y_train)

y_pred=spam_detect_model.predict(X_test)

In [41]:
X_test.shape

(1115, 7094)

In [28]:
y_pred

array([0, 0, 0, ..., 0, 1, 0], dtype=uint8)

In [29]:
from sklearn.metrics import confusion_matrix
confusion_m = confusion_matrix(y_test,y_pred)


In [30]:
confusion_m

array([[955,   0],
       [ 31, 129]], dtype=int64)

In [31]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test,y_pred)

In [32]:
accuracy

0.9721973094170404

In [126]:
input_sentence = "WINNER!! URGENT! We are trying to contact you. a £900 prize reward! Valid 12 hours only. This is not SPAM"
preprocessed_sentence = preprocess([input_sentence])

In [127]:
preprocessed_sentence

['winner urgent trying contact prize reward valid hour spam']

In [128]:
test = cv.transform(preprocessed_sentence).toarray()

In [129]:
np.unique(test)

array([0.        , 0.26784848, 0.27901624, 0.27975421, 0.29422803,
       0.3039102 , 0.32742521, 0.35338576, 0.38099226, 0.46472497])

In [130]:
test.shape

(1, 7094)

In [131]:
predicted_label = spam_detect_model.predict(test)

if predicted_label[0] == 1:
    print("The sentence is spam.")
else:
    print("The sentence is not spam.")


The sentence is spam.
